In [1]:
# Objects for defining the Hamiltonian
from PySW import RDSymbol, RDBasis, BosonOp, Dagger, Block
# Objects for obtaining the Effective Hamiltonian
from PySW import EffectiveFrame
# Built-in symbols
from PySW import hbar, t
# Useful functions
from PySW import display_dict, group_by_operators

import sympy as sp

# Setup

In [2]:
# ---------------- Defining the symbols ------------------
# Order 0
omega = RDSymbol('omega', positive=True, real=True)
omega_z = RDSymbol('omega_z', positive=True, real=True)
omega_d = RDSymbol('omega_d', positive=True, real=True)

# Order 1
g = RDSymbol('g', order=1, positive=True, real=True)
E0 = RDSymbol('E0', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Spin basis: Finite 2x2 Hilbert space
Spin = RDBasis('sigma', dim=2)
s0, sx, sy, sz = Spin.basis
# Boson basis: Infinite bosonic Hilbert space
a = BosonOp('a')
ad = Dagger(a)

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0 = omega * ad * a - sp.Rational(1, 2) * omega_z * sz
# Perturbation Hamiltonians
V = g * (ad + a) * sx
Hd = E0 * sp.cos(omega_d * t) * (a + ad)

display_dict({
    sp.Symbol('H_0'): H0,
    sp.Symbol('V'): V,
    sp.Symbol('H_d'): Hd
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Time Independent and then rotate

In [3]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [4]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=True, mask=None)
# Obtaining the result in the operator form
H_eff_SWT = Eff_frame.get_H(return_form='operator')
H_eff_SWT

The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  8.79it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████████| 2/2 [00:00<00:00, 55.27it/s]


-g**2*omega/(omega**2 - omega_z**2) + g**2*omega_z*sigma_3/(omega**2 - omega_z**2) + 2*g**2*omega_z*sigma_3*Dagger(a)*a/(omega**2 - omega_z**2) + omega*Dagger(a)*a - omega_z*sigma_3/2

### Rotate Drive

In [5]:
H_Drive = Eff_frame.rotate(Hd)
H_Drive

Projecting to operator form: 100%|██████████| 3/3 [00:00<00:00, 230.86it/s]


-2*E0*g*omega*cos(omega_d*t)*sigma_1/(omega**2 - omega_z**2) + E0*cos(omega_d*t)*Dagger(a) + E0*cos(omega_d*t)*a

In [6]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}
H_qubit_TI = (H_eff_SWT + H_Drive).xreplace(subs_expvals)
H_qubit_TI

-2*E0*g*omega*cos(omega_d*t)*sigma_1/(omega**2 - omega_z**2) - g**2*omega/(omega**2 - omega_z**2) + g**2*omega_z*sigma_3/(omega**2 - omega_z**2) - omega_z*sigma_3/2

# Time Dependent

In [7]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0, V + Hd, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [12]:
# Defining the mask
mask = Block(fin=sx, inf=a, subspaces=[Spin]) + Block(fin=s0, inf=a, subspaces=[Spin]) + Block(fin=s0, inf=a**2, subspaces=[Spin])
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=mask)
# Obtaining the result in the operator form
H_eff_SWT = Eff_frame.get_H(return_form='matrix')
H_eff_SWT

The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  6.73it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to matrix form: 100%|██████████| 2/2 [00:00<00:00, 982.39it/s]


Matrix([
[-g**2*omega/((-omega - omega_z)*(omega + omega_z)) - g**2*omega_z/((-omega - omega_z)*(omega + omega_z)) - g**2/(omega + omega_z) + g**2/(-omega - omega_z) - I*hbar*(-(-E0*exp(I*omega_d*t)/(2*(hbar*omega_d - omega)) - E0*exp(-I*omega_d*t)/(2*(-hbar*omega_d - omega)))*(-I*E0*omega_d*exp(I*omega_d*t)/(2*(hbar*omega_d + omega)) + I*E0*omega_d*exp(-I*omega_d*t)/(2*(-hbar*omega_d + omega)))/2 + (-E0*exp(I*omega_d*t)/(2*(hbar*omega_d + omega)) - E0*exp(-I*omega_d*t)/(2*(-hbar*omega_d + omega)))*(-I*E0*omega_d*exp(I*omega_d*t)/(2*(hbar*omega_d - omega)) + I*E0*omega_d*exp(-I*omega_d*t)/(2*(-hbar*omega_d - omega)))/2) - omega*(-E0*exp(I*omega_d*t)/(2*(hbar*omega_d - omega)) - E0*exp(-I*omega_d*t)/(2*(-hbar*omega_d - omega)))*(-E0*exp(I*omega_d*t)/(2*(hbar*omega_d + omega)) - E0*exp(-I*omega_d*t)/(2*(-hbar*omega_d + omega))) + omega*Dagger(a)*a - omega_z/2 - (E0*exp(I*omega_d*t)/2 + E0*exp(-I*omega_d*t)/2)*(-E0*exp(I*omega_d*t)/(2*(hbar*omega_d - omega)) - E0*exp(-I*omega_d*t)/(2*(-hb

In [13]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}
H_qubit_TD = H_eff_SWT.xreplace(subs_expvals)

In [36]:
group_by_operators(Spin.project(sp.expand_complex(sp.factor_terms(H_qubit_TD.expand())).trigsimp()).subs(hbar, 1))[sx].factor()

-E0*g*omega*(2*omega**2 - omega_d**2 - omega_z**2)*cos(omega_d*t)/((omega - omega_d)*(omega + omega_d)*(omega - omega_z)*(omega + omega_z))

In [39]:
omega_driving_resonance = group_by_operators(Spin.project(sp.expand_complex(sp.factor_terms(H_qubit_TD.expand())).trigsimp()).subs(hbar, 1))[sz]*2
omega_driving_resonance.factor()

omega_z*(2*g**2 - omega**2 + omega_z**2)/((omega - omega_z)*(omega + omega_z))

In [46]:
group_by_operators(Spin.project(sp.expand_complex(sp.factor_terms(H_qubit_TD.expand())).trigsimp()).subs(hbar, 1))[sx].factor().coeff(sp.cos(omega_d*t)).subs(omega_d, omega_driving_resonance).factor()

-2*E0*g*omega*(2*g**4*omega_z**2 - 2*g**2*omega**2*omega_z**2 + 2*g**2*omega_z**4 - omega**6 + 3*omega**4*omega_z**2 - 3*omega**2*omega_z**4 + omega_z**6)/((omega - omega_z)*(omega + omega_z)*(2*g**2*omega_z - omega**3 - omega**2*omega_z + omega*omega_z**2 + omega_z**3)*(2*g**2*omega_z + omega**3 - omega**2*omega_z - omega*omega_z**2 + omega_z**3))

In [48]:
(1/(omega**2 - (omega_driving_resonance)**2)).simplify().factor()

-(omega - omega_z)**2*(omega + omega_z)**2/((2*g**2*omega_z - omega**3 - omega**2*omega_z + omega*omega_z**2 + omega_z**3)*(2*g**2*omega_z + omega**3 - omega**2*omega_z - omega*omega_z**2 + omega_z**3))